## 第6章 実務に役立つアルゴリズムを知る  
#### keyword:

### 6.1 最短経路問題とは？

時間・費用・距離などをそれぞれ数値化してコストとして考える。

格子状の経路で、最短経路がいくつあるか求める手法
1. 右にm回, 上にn回移動する場合m+nCm
2. 交点を通るパターン数を左下から足していく → プログラミングでも便利  
- 動的計画法（DP）

In [ ]:
# near_route1.py

M, N = 6, 5

route = [[0 for i in range(N + 1)] for j in range(M + 1)]

for i in range(M + 1):
    route[i][0] = 1
    
for i in range(1, N + 1):
    route[0][i] = 1
    for j in range(1, M + 1):
        route[j][i] = route[j - 1][i] + route[j][i - 1]
        
print(route[M][N])

- メモ化(計算済の情報を保持)

In [ ]:
# near_route2.py

import functools

M, N = 6, 5

@functools.lru_cache(maxsize = None)
def search(m, n):
    if (m == 0) or (n == 0):
        return 1
    
    return search(m - 1, n) + search(m, n - 1)

print(search(M, N))

### *20210205*

### 6.2 ベルマンフォード法

- 辺の重みに注目して解く  
- スタート地点は0, それ以外のコストの初期値は無限大`float('inf')`  

In [ ]:
# bellman_ford.py

def bellman_ford(edges, num_v):
    dist = [float('inf') for i in range(num_v)]
    dist[0] = 0
    
    changed = True #コストが更新されたか
    while changed:
        changed = False
        for edge in edges:
            if dist[edge[1]] > dist[edge[0]] + edge[2]:
                dist[edge[1]] = dist[edge[0]] + edge[2]
                changed = True
                
    return dist

# １つの辺は起点と終点の番号、コストの３つの要素をもつ
edges = [
    [0, 1, 4], [0, 2, 3], [1, 2, 1], [1, 3, 1],
    [1, 4, 5], [2, 5, 2], [4, 6, 2], [5, 4, 1],
    [5, 6, 4]
]

print(bellman_ford(edges, 7))

### 6.3 ダイクストラ法

コストが最小になる頂点を選択することを繰り返す

In [ ]:
# dijkstra.py

def dijkstra(edges, num_v):
    dist = [float('inf')] * num_v
    dist[0] = 0
    q = [i for i in range(num_v)]
    
    while len(q) > 0:
        r = q[0]
        for i in q:
            if dist[i] < dist[r]:
                r = i
                
        u = q.pop(q.index(r))
        for i in edges[u]:
            if dist[i[0]] > dist[u] + i[1]:
                dist[i[0]] = dist[u] + i[1]
                
    return dist

edges = [
    [[1, 4], [2, 3]],
    [[2, 1], [3, 1], [4, 5]],
    [[5, 2]],
    [[4, 3]],
    [[6, 2]],
    [[4, 1], [6, 4]],
    []
]

print(dijkstra(edges, 7))

### *20210206*

### 6.4 A*アルゴリズム

- ゴールまでのコストの推定値を取り入れる方法。
- 以下ではマンハッタン距離を推定コストに使う場合を考える。

### 6.5 文字列探索の力任せ法

長い文章の中から特定の文字列を探す
- テキストとパターンが１文字ずつ一致するか調べる

In [ ]:
# search_string.py

text = list('SHOEISHA SESHOP')
pattern = list('SHA')

for i in range(len(text)):
    match = True
    for j in range(len(pattern)):
        if text[i + j] != pattern[j]:
            match = False
            break
            
    if match:
        print(i)
        break

### 6.6 Boyer-Moore法

パターンに含まれる文字と含まれない文字を区別して文字数をずらして探索。

In [ ]:
# search_string_bm.py

text = list('SHOEISHA SESHOP')
pattern = list('SHA')

skip = {}
for i in range(len(pattern) - 1):
    skip[pattern[i]] = len(pattern) - i - 1
    
print(skip)
    
i = len(pattern) - 1
while i < len(text):
    match = True
    for j in range(len(pattern)):
        if text[i - j] != pattern[len(pattern) - 1 - j]:
            match = False
            break
    if match:
        print(i - len(pattern) + 1)
        break
    if text[i] in skip:
        i += skip[text[i]]
    else:
        i += len(pattern)

### 6.7 逆ポーランド記法

- 数式の演算子を前に置く「ポーランド記法」、演算子を後ろに置く「逆ポーランド記法」。  
- 数の区別をする区切り文字は一般的にスペース。  
- 逆ポーランド記法で書かれたものはスタック(最後に格納したデータから順に取り出すデータ構造(5.5 ヒープソート参照))で処理しやすい。

In [ ]:
# calc.py

def calc(expression):
    stack = []
    for i in expression.split(' '):
        print(stack)
        if i == '+':
            b, a = stack.pop(), stack.pop()
            stack.append(a + b)
        elif i == '-':
            b, a = stack.pop(), stack.pop()
            stack.append(a - b)
        elif i == '*':
            b, a = stack.pop(), stack.pop()
            stack.append(a * b)
        elif i == '/':
            b, a = stack.pop(), stack.pop()
            stack.append(a // b)
        else:
            stack.append(int(i))
    return stack[0]

print(calc('4 6 2 + * 3 1 - 5 * -'))

### 6.8 ユークリッドの互除法

aをbで割った時の商をq、余りをrとする。
a = bq + r  
「aとbの最大公約数」は「bとrの最大公約数」に等しい。

In [ ]:
# gcd1.py

def gcd(a, b):
    r = a % b
    while r != 0:
        a, b = b, r
        r = a % b
        
    return b

print(gcd(1274, 975))

In [ ]:
# gcd2.py

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
        
    return a

print(gcd(1274, 975))

第6章 理解度check

In [ ]:
data = list('000000111111100111000000001111')

def compress(data):
    flag = 0 #0か１なのでフラグで管理
    cnt = 0
    result = []
    for i in data:
        if int(i) == flag:
            cnt += 1
        else:
            result.append(cnt)
            cnt = 1
            flag = 1 - flag
    result.append(cnt)
    return result
    
print(compress(data))